In [ ]:
from keras.utils.generic_utils import get_custom_objects
from custom_metrics import f1, precision, recall
import keras
from keras.preprocessing.image import ImageDataGenerator
import json


get_custom_objects().update({"f1": f1, 'precision': precision, 'recall': recall})

### Writing class files and loading model

In [ ]:
# VGG-16 and MobileNet can be tested out of the box
#For testing ResNet further preprocessing is needed refer to RESNET notebook

model = keras.models.load_model('./2Dense-255class-VGG-16-improvement_0.92.hdf5')

validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = validation_datagen.flow_from_directory(
        './Data/validation',  # This is the source directory for training images
        target_size=(224, 224),  # All images will be resized to 150x150
        batch_size=64,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical',
        shuffle=True)

In [ ]:
classes_ = list(validation_generator.class_indices.keys())
num_to_label = { value : key for key, value in validation_generator.class_indices.items()}

with open('classes.json', 'w') as f:
    json.dump(num_to_label, f)

with open('classes.json') as f:
    num_to_label = json.load(f)

num_to_label

### Preparing for prediction

In [ ]:
from keras.utils.generic_utils import get_custom_objects
from custom_metrics import f1, precision, recall
from keras.preprocessing.image import img_to_array
# convert the image pixels to a numpy array
from keras.preprocessing.image import load_img
import numpy as np
# load an image from file
from keras.models import load_model
import json



with open('classes.json') as f:
    num_to_label = json.load(f)

def predict_label_from_file(image, class_labels='classes.json'):
    "jpeg file path and json file containing index -> name as key value pairs"
    
    
    image = load_img(image, target_size=(224, 224))
    image = img_to_array(image) / 255
    image = np.expand_dims(image, axis=0)
    prediction_tensor = model.predict_on_batch(image)
    most_likely = np.argmax(prediction_tensor)
    
    return num_to_label[str(most_likely)]


In [ ]:
predict_label_from_file('zed.jpg', num_to_label)

In [ ]:
# Predict from generator

validation_generator.reset()
actual_lables = [num_to_label[clazz] for clazz in validation_generator.classes]
actual_class = validation_generator.classes

pred=model.predict_generator(validation_generator,verbose=1,steps=validation_generator.samples/64)
predicted_label = [classes_[np.argmax(i)] for i in pred]
predicted_class = [np.argmax(i) for i in pred]
